In [2]:
from scipy.stats import wilcoxon
import pandas as pd

# Metrics from Model A
model_a = {
    "Accuracy": [0.4892, 0.5252, 0.5036, 0.5540, 0.5870, 0.5290, 0.4855, 0.4855],
    "Precision": [0.0000, 0.0000, 0.0000, 0.4333, 0.6667, 0.0000, 0.0000, 0.3158],
    "Recall": [0.0000, 0.0000, 0.0000, 0.2241, 0.0678, 0.0000, 0.0000, 0.0938],
    "F1 Score": [0.0000, 0.0000, 0.0000, 0.2955, 0.1231, 0.0000, 0.0000, 0.1446]
}

# Metrics from Model B
model_b = {
    "Accuracy": [0.5540, 0.5396, 0.5971, 0.5755, 0.6043, 0.6304, 0.5942, 0.6159],
    "Precision": [0.5000, 0.5882, 0.5714, 0.5217, 0.5714, 0.6000, 0.5362, 0.5714],
    "Recall": [0.6452, 0.5263, 0.7059, 0.5806, 0.6154, 0.6462, 0.6066, 0.6349],
    "F1 Score": [0.5634, 0.5556, 0.6316, 0.5496, 0.5926, 0.6222, 0.5692, 0.6015]
}

# Run Wilcoxon test for each metric
results = {}
for metric in model_a:
    stat, pval = wilcoxon(model_a[metric], model_b[metric], alternative='less')
    results[metric] = {"Wilcoxon Statistic": stat, "p-value": pval}

# Convert to DataFrame for display
results_df = pd.DataFrame(results).T

results_df

,Wilcoxon Statistic,p-value
Accuracy,0.0,0.003906
Precision,2.0,0.011719
Recall,0.0,0.003906
F1 Score,0.0,0.003906
